In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
import pandas as pd

import pygame as gui
import sys
import torch
import speech_recognition as sr
from transformers import BertTokenizer

w,h = 1600,900
white = (255,255,255)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 128

class TransformerModel(nn.Module):
    def __init__(self, num_classes):
        super(TransformerModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(768, num_classes)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs['pooler_output']
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return logits

model = TransformerModel(2)
model.load_state_dict("sentiment.pth")
model.eval()

class TomCounts_ARI:
	def __init__(self):
		gui.mixer.init()
		gui.init()
		gui.font.init()
		self.det = None
		self.list = []
		self.font = gui.font.Font("./quake.TTF", 80)
		self.count = 0
		self.width = 1920
		self.height = 1080
		self.bgimg = gui.image.load('bg.png')
		self.logo = gui.image.load('logo.png')
		self.shinelogo = gui.image.load('bglogo.png')
		self.shinelogo = gui.transform.scale(self.shinelogo,(800,800))
		self.bgimg = gui.transform.scale(self.bgimg,(self.width//2-100,self.height))
		self.disp = gui.display.set_mode((self.width,self.height),0,0)
		gui.display.set_caption("SENTIMENT Tom")
		self.img = gui.image.load("Speak/0001.jpg")
		self.img = gui.transform.scale(self.img,(w+120,h+160))
		self.CAPTURE_ALL()
    
	def infer(self,text):
	    model.eval()
	    encoded_input = tokenizer.encode_plus(
	        text,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
	    input_ids = encoded_input['input_ids']
	    attention_mask = encoded_input['attention_mask']
        
	    with torch.no_grad():
	        outputs = model(input_ids, attention_mask)
	        _, predicted_labels = torch.max(outputs, dim=1)
	        return predicted_labels.item()

	def recordAudio(self):
		r = sr.Recognizer()
		r.dynamic_energy_threshold = False
		r.energy_threshold = 480
		with sr.Microphone() as source:
		    r.adjust_for_ambient_noise(source=source)
		    print('Listening')
		    try:
			    audio = r.listen(source,timeout=4,phrase_time_limit=4)
			    data = r.recognize_google(audio)
			    print(self.infer(data))
			    
		    except:
		        print("ERROR")

	def blitForever(self,val=None):
		if(val!=None):
			if(val not in self.list):
				self.list = []
				self.list.append(val)
				self.playAudioARI(val)
		else:
			self.disp.blit(self.img,(560,0,0,0))
		self.disp.blit(self.bgimg,(0,0))
		self.disp.blit(self.shinelogo,(0,self.height//2-450))
		self.disp.blit(self.logo,(150,self.height//2-300))
		text1 = self.font.render("TOM DETECTED "+str(self.det), True, white)
		text1Rect = text1.get_rect()
		text1Rect.center = (w//2,h)
		self.blittext()
		gui.display.update()


	def blittext(self):
		text1 = self.font.render("TOM DETECTED "+str(self.det), True, white)
		text1Rect = text1.get_rect()
		text1Rect.center = (w//2+600,h+40)
		self.disp.blit(text1,text1Rect)
 
	def playAudioARI(self,op):
		i = 1
		self.det = op[9]
		gui.mixer.music.load(op)
		gui.mixer.music.play()
		while(True):
			img = gui.image.load("Speak/0"+str(i).zfill(3)+".jpg")
			img = gui.transform.scale(img,(w+120,h+160))
			self.disp.blit(img,(560,0,0,0))
			i+=1
			if(i==10):
				return
			self.disp.blit(self.imgframe,(0,0,0,0))
			self.blittext()
			gui.display.update()


	def CAPTURE_ALL(self):
		RUN = True
		while RUN:
			self.blitForever()
			for	eve in gui.event.get():
				if eve.type==gui.KEYDOWN:
					if eve.key==gui.K_RETURN:
						self.infer("I am good")
					elif eve.key==gui.K_ESCAPE:
						RUN = False
						gui.quit()
						return

TomCounts_ARI()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


TypeError: Expected state_dict to be dict-like, got <class 'str'>.